In [ ]:
# MIT License

# Copyright (c) 2025 André M. P. Mattos, Douglas A. Santos

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Processing Radiation Reliability Metric
> Example calculations for PRRM on PolarFire SoC

The PRRM scores are highly based on existing concepts, such as the cross section and confidence margins, and the performance reported by Embench. When reporting the PRRM, each score should be present as shown in 1., for each risk level (low, medium, and high), and obtained as 2.: 

1. $ PRRM_{<risk\_level>} \pm (PRRM_{err\_lower}, PRRM_{err\_upper}) $

2. $ PRRM_{<risk\_level>} = \frac{\sigma_{<risk\_level>}}{EPM} \times 10^{12} $  

Where $\sigma_{<risk\_level>}$ 3. is the cross section for the specific risk level and $EPM$ 4. is the Embench Performance Metric, which represents the geometric average of the relative performance of the target processor with a reference processor: 

3. $\sigma_{<risk\_level>} = \frac{N_{err~<risk\_level>}}{fluence}$

4. $EPM = \left(\prod_{i=1}^{n} \frac{WPM_{ref_i}}{WPM_i} \right)^{1/n} \times n_{parallel}$   

Where $n$ is the number of workloads, WPM stands for Workload Performance Metric 5., $ref$ is the reference platform results and parameters provided by Embench. Also, we added the $n_{parallel}$ term to the original Embench, which is the number of workloads in parallel (accounting for multi-core systems), but it is defined as 1 in this work due to a single-core benchmark implementation. $W_{time}$ is the obtained execution time, and $CPU_{freq}$ and $SF$ (Scaling Factor).

5. $WPM_i = \frac{W_{time_{~i}}}{SF_{i} \times CPU_{freq_{~i}}}$   

The $PRRM_{err\_lower}$ and $PRRM_{err\_upper}$ are calculated based on the uncertainty of the cross section since we assume for simplicity that EPM variation is negligible for radiation testing purposes. These error margins represent a 95\% confidence interval for a 10\% beam fluence uncertainty in the cross sections. Finally, we added a $10^{12}$ factor to provide scores with a more compact presentation. A lower PRRM score represents a better result as inherited by the cross section and reinforced by the EPM, which as higher, more computing performance. Therefore, it can be loosely interpreted as the probability of errors per risk level group normalized by the performance.


In [12]:
# Performance scores and scaling factors of Embench's reference platform (Cortex-M4) and the example platform (PolarFire SoC, appcore) 

# Ref platform specs
ref_platform_freq = 16     #16MHz
ref_platform_speed = { 
    "aha-mont64"    : 4004,
    "crc32"         : 4010,
    "cubic"         : 3931,
    "edn"           : 4010,
    "matmult"       : 3985,
    "minver"        : 3998,
    "nbody"         : 2808,
    "nettle-aes"    : 4026,
    "nettle-sha256" : 3997,
    "nsichneu"      : 4001,
    "slre"          : 4010,
    "st"            : 4080,
    "statemate"     : 4001,
    "ud"            : 3999,
    "wikisort"      : 2779
}
ref_platform_scaling = { 
    "aha-mont64"    : 423,
    "crc32"         : 170,
    "cubic"         : 10,
    "edn"           : 87,
    "matmult"       : 46,
    "minver"        : 555,
    "nbody"         : 1,
    "nettle-aes"    : 78,
    "nettle-sha256" : 475,
    "nsichneu"      : 1231,
    "slre"          : 110,
    "st"            : 13,
    "statemate"     : 1964,
    "ud"            : 1478,
    "wikisort"      : 1
}

# MPFS specs
mpfs_platform_freq = 600     #600MHz
mpfs_platform_speed = { 
    "aha-mont64"    : 3865,
    "crc32"         : 4086,
    "cubic"         : 4075,
    "edn"           : 4054,
    "matmult"       : 4559,
    "minver"        : 4017,
    "nbody"         : 3805,
    "nettle-aes"    : 4241,
    "nettle-sha256" : 4029,
    "nsichneu"      : 4083,
    "slre"          : 3994,
    "st"            : 4031,
    "statemate"     : 4092,
    "ud"            : 3963,
    "wikisort"      : 4816
}
mpfs_platform_scaling = { 
    "aha-mont64"    : 698,
    "crc32"         : 31,
    "cubic"         : 12,
    "edn"           : 11,
    "matmult"       : 5,
    "minver"        : 326,
    "nbody"         : 3,
    "nettle-aes"    : 7,
    "nettle-sha256" : 146,
    "nsichneu"      : 229,
    "slre"          : 27,
    "st"            : 85,
    "statemate"     : 469,
    "ud"            : 333,
    "wikisort"      : 1 
}

In [2]:
%pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 52.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 64.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
from scipy.stats import gmean

wpm_relative = []

for workload in ref_platform_speed.keys():
    wpm_relative.append((ref_platform_speed[workload]/(ref_platform_scaling[workload]*ref_platform_freq))/ (mpfs_platform_speed[workload]/(mpfs_platform_scaling[workload]*mpfs_platform_freq)))
    
print(wpm_relative)

epm = gmean(wpm_relative)
print(epm)

# Just for checking, these are the reported CoreMark/MIPS scores reported by the manufacturers 
# PolarFire SoC: 3.125 CoreMarks/MHz, 1.714DMIPS/MHz
# Reference platform: 3.42 CoreMark/MHz, 1.25 DMIPS/MHz

# Now, as a comparison:
# Coremark Score (best evaluation from manufacturers): Application cores of PolarFire SoC are 34.27 times faster than Cortex-M4 (considering freq)
# Our example (embench with -Os optimization and not tuned): Application cores of PolarFire SoC are 15.18 times faster than Cortex-M4 (considering freq)

[64.10485077023296, 6.711043448216292, 43.40981595092025, 4.689918854090468, 3.5628880284578046, 21.922841437404543, 83.02233902759527, 3.1947744545009336, 11.434768977544383, 6.835934119259325, 9.241418946601721, 248.1728145335191, 8.755794364544201, 8.52566765244274, 21.63880813953488]
15.183419383167744


### Get the number of errors for high, medium, and low risks 
> Considering the PolarFire SoC example with high-energy protons experiment

[embench-singlecore]
Total: 26 SEFIs
- high-risk: 
  - hangs: 13
  - mismatch: 2 (diagnosis: unknown)
- medium-risk:
  - mismatch: 1 (diagnosis: l2_cache)
  - wdt_bad: 0
- low-risk:
  - wdt_good: 0

[embench-freertos]
Total: 9 SEFIs
- high-risk: 
  - hangs: 2 
  - mismatch: 1 (diagnosis: unknown)
- medium-risk:
  - mismatch: 1 (diagnosis: l2_cache/tilelink)
  - wdt_bad: 1
- low-risk:
  - wdt_good: 1

In [14]:
# Accumulated fluence during the experiment (after filtering) 

singlecore_fluence = 4.737303e+11
freertos_fluence = 1.333344e+11

print(f'singlecore_fluence: {singlecore_fluence:2e}')
print(f'freertos_fluence: {freertos_fluence:2e}')


singlecore_fluence: 4.737303e+11
freertos_fluence: 1.333344e+11


In [15]:
# Calculate confidence margins
# Author: Daniel Söderström

import numpy as np

# Parameters (Can be sent to the functions also if wanted)
BEAM_UNCERTAINTY = 0.1
BITS = 4*1024*1024
UNCERTAINTY_TABLE = np.loadtxt('./error_table_25100.txt', skiprows=2)

# The table is from the ESA guidelines (ESCC 25100) how to calculate the error bars. There is certainly cleaner ways to do this, 
# but it would involve some more advanced math probably, which I haven't had the patience to go through yet

# Calculate the upper and lower confidence limits (95 %) for a particle run with Nevents upsets
# Predefine number of bits BITS, lookup table UNCERTAINTY_TABLE, and beam uncertainty BEAM_UNCERTAINTY
# Calculate the differential uncertainty levels
def differentialUncertaintyLevels(Nevents, beam_uncertainty = BEAM_UNCERTAINTY):
    # delta_upper / _lower is the variance on the measured number of events.
    if Nevents > 9999:
        # Set the number of events to 10000 (max in table), it will be
        #    small error bars either way, dominated by the beam uncertainty
        Nevents = 10000

    if Nevents == 0:
        lower_limit = 0

        delta_upper = UNCERTAINTY_TABLE[:, 4][0]
        upper_limit = np.sqrt(delta_upper**2 + beam_uncertainty**2)
    elif Nevents in UNCERTAINTY_TABLE[:, 0]:
        # Lower limit
        delta_upper = UNCERTAINTY_TABLE[:, 3][UNCERTAINTY_TABLE[:, 0] == Nevents]
        lower_limit = np.sqrt((delta_upper/Nevents)**2 + beam_uncertainty**2)

        # Upper limit
        delta_upper = UNCERTAINTY_TABLE[:, 4][UNCERTAINTY_TABLE[:, 0] == Nevents]
        upper_limit = np.sqrt((delta_upper/Nevents)**2 + beam_uncertainty**2)
    else:
        # Lower limit
        DeltaBelow = UNCERTAINTY_TABLE[:, 3][UNCERTAINTY_TABLE[:, 0] < Nevents]
        NBelow = UNCERTAINTY_TABLE[:, 0][UNCERTAINTY_TABLE[:, 0] < Nevents]
        DeltaAbove = UNCERTAINTY_TABLE[:, 3][len(DeltaBelow)]
        NAbove = UNCERTAINTY_TABLE[:, 0][len(DeltaBelow)]
        DeltaBelow = DeltaBelow[-1]
        NBelow = NBelow[-1]

        # Assume piecewise linear behaviour, delta_ = k*Nevents + m
        k = (DeltaAbove - DeltaBelow) / (NAbove - NBelow)
        delta_lower = DeltaBelow + k*(Nevents - NBelow)
        lower_limit = np.sqrt((delta_lower/Nevents)**2 + beam_uncertainty**2)

        # Upper limit
        DeltaBelow = UNCERTAINTY_TABLE[:, 4][UNCERTAINTY_TABLE[:, 0] < Nevents]
        NBelow = UNCERTAINTY_TABLE[:, 0][UNCERTAINTY_TABLE[:, 0] < Nevents]
        DeltaAbove = UNCERTAINTY_TABLE[:, 4][len(DeltaBelow)]
        NAbove = UNCERTAINTY_TABLE[:, 0][len(DeltaBelow)]
        DeltaBelow = DeltaBelow[-1]
        NBelow = NBelow[-1]

        # Assume piecewise linear behaviour, delta_ = k*Nevents + m
        k = (DeltaAbove - DeltaBelow) / (NAbove - NBelow)
        delta_upper = DeltaBelow + k*(Nevents - NBelow)
        upper_limit = np.sqrt((delta_upper/Nevents)**2 + beam_uncertainty**2)
    
    try:
        if len(lower_limit):
            lower_limit = lower_limit.tolist()[0]
    except:
        pass

    try:
        if len(upper_limit):
            upper_limit = upper_limit.tolist()[0]
    except:
        pass

    return lower_limit, upper_limit

# Calculate the cross section (this can take arrays)
def crossSection(Nevents, fluence, bits=BITS):
    return Nevents/(fluence * bits)

# Calculate the actual uncertainty levels (suitable to e.g. plug into yerr in pyplots errorbar plot)
# Function of number of events Nevents and fluence fluence
def errorBars(Nevents, fluence, bits = BITS):
    XS = crossSection(Nevents, fluence, bits=bits)
    diff_lower, diff_upper = differentialUncertaintyLevels(Nevents)
    if Nevents == 0:
        lower_level = 0
        if fluence != 0:
            upper_level = diff_upper/(fluence * bits)
        else:
            upper_level = 0
    else:
        lower_level = XS * diff_lower
        upper_level = XS * diff_upper

    return lower_level, upper_level

In [16]:
singlecore_highrisk = 15
singlecore_mediumrisk = 1
singlecore_lowrisk = 0

freertos_highrisk = 3
freertos_mediumrisk = 2
freertos_lowrisk = 1

prrm_factor = 1e12

def prrm_score(Nevents, fluence):
    prrm = (crossSection(Nevents, fluence, 1) / epm) * prrm_factor
    diff_lower, diff_upper = differentialUncertaintyLevels(Nevents)
    if Nevents == 0:
        lower_level = 0
        if fluence != 0:
            upper_level = (prrm_factor/epm) * (diff_upper/fluence)
        else:
            upper_level = 0
    else:
        lower_level = prrm * diff_lower
        upper_level = prrm * diff_upper

    return prrm, lower_level, upper_level

prrm_singlecore_high = prrm_score(singlecore_highrisk, singlecore_fluence)
prrm_singlecore_medium = prrm_score(singlecore_mediumrisk, singlecore_fluence)
prrm_singlecore_low = prrm_score(singlecore_lowrisk, singlecore_fluence)

prrm_freertos_high = prrm_score(freertos_highrisk, freertos_fluence)
prrm_freertos_medium = prrm_score(freertos_mediumrisk, freertos_fluence)
prrm_freertos_low = prrm_score(freertos_lowrisk, freertos_fluence)

print('Embench singlecore scores:')
print(f'prrm_singlecore_high:\t {prrm_singlecore_high[0]:.2f} ± ({prrm_singlecore_high[1]:.2f}, {prrm_singlecore_high[2]:.2f})')
print(f'prrm_singlecore_medium:\t {prrm_singlecore_medium[0]:.2f} ± ({prrm_singlecore_medium[1]:.2f}, {prrm_singlecore_medium[2]:.2f})')
print(f'prrm_singlecore_low:\t {prrm_singlecore_low[0]:.2f} ± ({prrm_singlecore_low[1]:.2f}, {prrm_singlecore_low[2]:.2f})')
print('\nEmbench freertos scores:')
print(f'prrm_freertos_high:\t {prrm_freertos_high[0]:.2f} ± ({prrm_freertos_high[1]:.2f}, {prrm_freertos_high[2]:.2f})')
print(f'prrm_freertos_medium:\t {prrm_freertos_medium[0]:.2f} ± ({prrm_freertos_medium[1]:.2f}, {prrm_freertos_medium[2]:.2f})')
print(f'prrm_freertos_low:\t {prrm_freertos_low[0]:.2f} ± ({prrm_freertos_low[1]:.2f}, {prrm_freertos_low[2]:.2f})')

Embench singlecore scores:
prrm_singlecore_high:	 2.09 ± (0.93, 1.36)
prrm_singlecore_medium:	 0.14 ± (0.14, 0.64)
prrm_singlecore_low:	 0.00 ± (0.00, 0.51)

Embench freertos scores:
prrm_freertos_high:	 1.48 ± (1.18, 2.85)
prrm_freertos_medium:	 0.99 ± (0.87, 2.58)
prrm_freertos_low:	 0.49 ± (0.48, 2.26)


### Final Results
> PolarFire SoC PRRM Scores

Applying the PRRM to the results for high-energy proton irradiation, we obtain the scores for baremetal and FreeRTOS benchmarks. Notably, for FreeRTOS, due to the limited selection of workloads, it is less representative and is shown as a reference. 

##### PRRM Score of baremetal benchmark:

$PRRM_{high\_risk}$: 2.09 ± (0.93, 1.36)

$PRRM_{medium\_risk}$: 0.14 ± (0.14, 0.64)

$PRRM_{low\_risk}$: 0.00 ± (0.00, 0.51)

##### PRRM Score of FreeRTOS benchmark:
$PRRM_{high\_risk}$: 1.48 ± (1.18, 2.85)

$PRRM_{medium\_risk}$: 0.99 ± (0.87, 2.58)

$PRRM_{low\_risk}$: 0.49 ± (0.48, 2.26)

	 
	 
	 
	 
	 
	 
